# EJERCICIOS DEL CAPITULO 3

# PROBLEMA 1

## A

In [15]:
%%writefile add.cu

#include <iostream>
#include <math.h>

__global__
void sumape(int N, float *A, float *B, float *C)
{
  for (int i = 0; i < N; i++)
      C[i] = A[i] + B[i];
}

void hostStubFun(int N, float *A, float*B,float *C){
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  float *ptr_gpu_A , *ptr_gpu_B, *ptr_gpu_C;
  int sz = N*sizeof(float);
  cudaMalloc((void**)&ptr_gpu_A, sz);
  cudaMalloc((void**)&ptr_gpu_B, sz);
  cudaMalloc((void**)&ptr_gpu_C, sz);

  cudaMemcpy(ptr_gpu_A,A,sz,cudaMemcpyHostToDevice);
  cudaMemcpy(ptr_gpu_B,B,sz,cudaMemcpyHostToDevice);

  cudaMemcpy(ptr_gpu_C,C,sz,cudaMemcpyHostToDevice);

 cudaEventRecord(start);
  sumape<<<1,1>>>(N, ptr_gpu_A, ptr_gpu_B,ptr_gpu_C);
  cudaEventRecord(stop);

  cudaError_t error = cudaGetLastError();
  if(error != cudaSuccess) {
      printf("Error lanzando kernel: %s\n", cudaGetErrorString(error));
  }

  cudaMemcpy(C,ptr_gpu_C,sz,cudaMemcpyDeviceToHost);

  cudaEventSynchronize(stop);

  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("Tiempo transcurrido: %f ms\n", milliseconds);

  cudaFree(ptr_gpu_A);
  cudaFree(ptr_gpu_B);
  cudaFree(ptr_gpu_C);

 cudaEventDestroy(start);
cudaEventDestroy(stop);

}

int main(void)
{
  int N = 256;
  int N2 = N*N;
  float *C = new float[N2];
  float *A = new float[N2];
  float *B = new float[N2];
  for (int i = 0; i < N2; i++) {
    A[i] = 1.0f;
    B[i] = 2.0f;
  }
  hostStubFun(N2,A,B,C);
  printf("%f",C[1]);


}

Writing add.cu


In [16]:
%%shell

nvcc add.cu -o add_cuda -arch=sm_75
./add_cuda

Tiempo transcurrido: 7.155424 ms
3.000000

## B

In [13]:
%%writefile add2.cu

#include <iostream>
#include <math.h>


__global__
void sumape(int N, float *A, float *B, float *C)
{
  int id =blockIdx.x * blockDim.x + threadIdx.x;
  if (id <N)
      C[id] = A[id] + B[id];
}

void hostStubFun(int N, float *A, float*B,float *C){
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);
  float *ptr_gpu_A , *ptr_gpu_B, *ptr_gpu_C;
  int sz = N*sizeof(float);
  cudaMalloc((void**)&ptr_gpu_A, sz);
  cudaMalloc((void**)&ptr_gpu_B, sz);
  cudaMalloc((void**)&ptr_gpu_C, sz);


  cudaMemcpy(ptr_gpu_A,A,sz,cudaMemcpyHostToDevice);
  cudaMemcpy(ptr_gpu_B,B,sz,cudaMemcpyHostToDevice);

  cudaMemcpy(ptr_gpu_C,C,sz,cudaMemcpyHostToDevice);

  int hiloxblock = 256;
  int blockxgrid = (N+hiloxblock-1)/hiloxblock;

  cudaEventRecord(start);
  sumape<<<blockxgrid,hiloxblock>>>(N, ptr_gpu_A, ptr_gpu_B,ptr_gpu_C);
  cudaEventRecord(stop);

  cudaError_t error = cudaGetLastError();
  if(error != cudaSuccess) {
      printf("Error lanzando kernel: %s\n", cudaGetErrorString(error));
  }


  cudaMemcpy(C,ptr_gpu_C,sz,cudaMemcpyDeviceToHost);

  cudaEventSynchronize(stop);

  float ms = 0;
  cudaEventElapsedTime(&ms, start, stop);
  printf("Tiempo transcurrido: %f ms\n", ms);

  cudaFree(ptr_gpu_A);
  cudaFree(ptr_gpu_B);
  cudaFree(ptr_gpu_C);
  cudaEventDestroy(start);
cudaEventDestroy(stop);

}

int main(void)
{
  int N = 256;
  int N2 = N*N;
  float *C = new float[N2];
  float *A = new float[N2];
  float *B = new float[N2];

  // initialize x and y arrays on the host
  for (int i = 0; i < N2; i++) {
    A[i] = 1.0f;
    B[i] = 2.0f;
  }

  hostStubFun(N2,A,B,C);
  printf("%f",C[1]);

}

Writing add2.cu


In [14]:


!nvcc add2.cu -o add_cuda2 -arch=sm_75
!./add_cuda2

Tiempo transcurrido: 0.075488 ms
3.000000

## C hilo x fila

In [11]:
%%writefile add3.cu
#include <iostream>
#include <math.h>

__global__
void sumape(int N, float *A, float *B, float *C){
  int id = blockIdx.x * blockDim.x + threadIdx.x;

  if(id < N){

    int i0 = id * N;
    for (int i = i0; i < i0 + N; i++)
      C[i]= A[i] + B[i];
  }
}

void hostStubFun(int N, float *A, float *B, float *C){
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  int N2 = N*N;
  float *ptr_gpuA, *ptr_gpuB, *ptr_gpuC;

  int sz = N2*sizeof(float);
  cudaMalloc((void**)&ptr_gpuA, sz);
  cudaMalloc((void**)&ptr_gpuB, sz);
  cudaMalloc((void**)&ptr_gpuC, sz);

  cudaMemcpy(ptr_gpuA,A,sz,cudaMemcpyHostToDevice);
  cudaMemcpy(ptr_gpuB,B,sz,cudaMemcpyHostToDevice);

  int hiloxblock=256;
  int blockxgrid = (N + hiloxblock - 1)/hiloxblock;

cudaEventRecord(start);
  sumape<<<blockxgrid,hiloxblock>>>(N,ptr_gpuA, ptr_gpuB,ptr_gpuC);
cudaEventRecord(stop);

  cudaMemcpy(C,ptr_gpuC,sz,cudaMemcpyDeviceToHost);

  cudaEventSynchronize(stop);

  float ms = 0;
  cudaEventElapsedTime(&ms, start, stop);
  printf("%f ms\n", ms);


  cudaFree(ptr_gpuA);
  cudaFree(ptr_gpuB);
  cudaFree(ptr_gpuC);

  cudaEventDestroy(start);
cudaEventDestroy(stop);

}

int main(void){
  float *A,*B,*C;
  int N =255;
  int N2 = N*N;
  A = new float[N2];
  B = new float[N2];
  C = new float[N2];
  for (int i = 0; i < N2; i++) {
    A[i] = 1.0f;
    B[i] = 2.0f;
  }
  hostStubFun(N,A,B,C);
  printf("%f",C[1]);
}



Overwriting add3.cu


In [12]:
!nvcc add3.cu -o add_cuda3 -arch=sm_75
!./add_cuda3

0.542720 ms
3.000000

## D hilo x columna

In [7]:
%%writefile add4.cu
#include <iostream>
#include <math.h>

__global__
void sumape(int N, float *A, float *B, float *C){
  int id = blockIdx.x * blockDim.x + threadIdx.x;

  if(id < N)
    for (int j = 0; j < N; j++)
      C[id+N*j]= A[id+N*j] + B[id+N*j];
}

void hostStubFun(int N, float *A, float *B, float *C){
  float *ptr_gpuA, *ptr_gpuB, *ptr_gpuC;
  int sz = N * N * sizeof(float);

  cudaMalloc(&ptr_gpuA, sz);
  cudaMalloc(&ptr_gpuB, sz);
  cudaMalloc(&ptr_gpuC, sz);

  cudaMemcpy(ptr_gpuA, A, sz, cudaMemcpyHostToDevice);
  cudaMemcpy(ptr_gpuB, B, sz, cudaMemcpyHostToDevice);

  int hilos = 256;
  int bloques = (N + hilos - 1) / hilos;

  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaEventRecord(start);
  sumape<<<bloques, hilos>>>(N, ptr_gpuA, ptr_gpuB, ptr_gpuC);
  cudaEventRecord(stop);

  cudaEventSynchronize(stop);

  float ms = 0.0f;
  cudaEventElapsedTime(&ms, start, stop);
  printf("%f ms\n", ms);

  cudaMemcpy(C, ptr_gpuC, sz, cudaMemcpyDeviceToHost);

  cudaEventDestroy(start);
  cudaEventDestroy(stop);

  cudaFree(ptr_gpuA);
  cudaFree(ptr_gpuB);
  cudaFree(ptr_gpuC);
}


int main(){
  int N = 255;
  int N2 = N * N;

  float *A = new float[N2];
  float *B = new float[N2];
  float *C = new float[N2];

  for(int i = 0; i < N2; i++){
    A[i] = 1.0f;
    B[i] = 2.0f;
  }

  hostStubFun(N, A, B, C);
  printf("%f\n", C[1]);
}




Overwriting add4.cu


In [8]:
!nvcc add4.cu -o add_cuda4 -arch=sm_75
!./add_cuda4

0.234720 ms
3.000000


# PROBLEMA 2

In [3]:
%%writefile mv.cu
#include <cuda_runtime.h>
#include <iostream>
#include <math.h>

__global__
void mvfun(int N, float *A, float *B, float *C) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < N) {
        float sum = 0.f;
        for (int j = 0; j < N; j++)
            sum += B[i*N + j] * C[j];
        A[i] = sum;
    }
}

void stubfun(int N, float *A,float *B,float *C)
{
    float *d_A, *d_B, *d_C;
    size_t vsz = N * sizeof(float);
    size_t msz = N * N * sizeof(float);

    cudaMalloc(&d_A, vsz);
    cudaMalloc(&d_B, msz);
    cudaMalloc(&d_C, vsz);

    cudaMemcpy(d_B, B, msz, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C, C, vsz, cudaMemcpyHostToDevice);

    int hiloxblock = 256;
    int blockxgrid = (N + hiloxblock - 1) / hiloxblock;

    mvfun<<<blockxgrid, hiloxblock>>>(N,d_A, d_B, d_C);

    cudaMemcpy(A, d_A, vsz, cudaMemcpyDeviceToHost);

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
}

int main() {
    const int N = 256;

    float B[N * N];
    float C[N];
    float A[N];

    for (int i = 0; i < N*N; i++) {
        B[i] = 1.0f;
    }

    for (int i = 0; i < N; i++) {
        C[i] = 1.0f;
    }

    stubfun(N, A, B, C);




    printf("%.1f\n",A[0]);


    return 0;
}

Overwriting mv.cu


In [4]:
!nvcc mv.cu -o mv -arch=sm_75
!./mv

256.0
